# Mount The drive

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**# Installing Transformer**

In [ ]:

!pip install transformers -U

In [ ]:
#pip uninstall transformers -y

In [ ]:
!pip install transformers==4.28.0

# Getting the data


In [3]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/Short Text Claasification/annotations_metadata.csv')

In [ ]:
df

,file_id,user_id,subforum_id,num_contexts,label
0,12834217_1,572066,1346,0,noHate
1,12834217_2,572066,1346,0,noHate
2,12834217_3,572066,1346,0,noHate
3,12834217_4,572066,1346,0,hate
4,12834217_5,572066,1346,0,noHate
...,...,...,...,...,...
10939,33676864_5,734541,1388,0,noHate
10940,33677019_1,735154,1388,0,noHate
10941,33677019_2,735154,1388,0,noHate
10942,33677053_1,572266,1388,0,hate


***Getting the text files they were stored differntly***

In [ ]:
folder_path = '/content/gdrive/MyDrive/Short Text Claasification/all_files'

In [ ]:
file_contents = []

import os

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            content = file.read()
            file_contents.append(content)

df_data = pd.DataFrame({'text': file_contents})


In [ ]:
df_data

In [ ]:
label_column = df['label']

In [ ]:
label_column

**Joining The all data**

In [ ]:
df_data['label'] = label_column

In [ ]:
replace_dict = {'noHate': 0, 'hate': 1}

# Replace the values in the 'label' column
df_data['label'] = df_data['label'].replace(replace_dict)

In [ ]:
df_data

,text,label
0,http://www.wntube.net/play.php ? vid = 1428 & ...,0
1,I was just trying to see how many of you saw o...,0
2,Please contribute and/or donate .,0
3,They have improved the site .,1
4,"People of the US are getting it together , fin...",0
...,...,...
10939,We have a couple of events coming up that are ...,0
10940,I feel its over .....,0
10941,Trudeau Senior got us into that mess .,0
10942,White men and women who have many white childr...,1


In [ ]:
data = df_data

In [ ]:
data.to_csv('/content/gdrive/MyDrive/Short Text Claasification/data.csv', index=False)


# Load the Intial Processed Data which was save in above line 

In [4]:
data = pd.read_csv('/content/gdrive/MyDrive/Short Text Claasification/data.csv')

In [ ]:
data

In [5]:
data =  data.rename(columns={'label': 'toxic'})

In [ ]:
data

In [ ]:
data['toxic'].value_counts()

In [ ]:
idk_skip_rows = data[data['toxic'] == 'idk/skip']
relation_rows = data[data['toxic'] == 'relation']

# Print the rows
print("Rows with 'idk/skip' label:")
print(idk_skip_rows)

print("\nRows with 'relation' label:")
print(relation_rows)

In [6]:
# Cleaining the data

# Filter out rows with 'idk/skip' and 'relation' labels
data = data[(data['toxic'] != 'idk/skip') & (data['toxic'] != 'relation')]


In [9]:
data['toxic'].value_counts()

0    9507
1    1196
Name: toxic, dtype: int64

**Checking The Type of the data , This was the root cause of one error which I encountered**

In [28]:
column_has_strings = data['toxic'].dtype == 'object'

In [29]:
print(column_has_strings )

True


In [30]:
string_values = data['toxic'].unique()
print(string_values)

['0' '1']


In [31]:
column_type = data['toxic'].dtype
print(column_type)

object


In [7]:
data['toxic'] = data['toxic'].astype(int)

Change the colum type

In [33]:
column_type = data['toxic'].dtype
print(column_type)

int64


# Due to GPU limitation the size of data is reduced 

In [ ]:
data = data[['text','toxic']]
data = data[0:1000]
data.head()

In [39]:
data['toxic'].value_counts()

0    890
1    110
Name: toxic, dtype: int64

**Importing some of the libraries **

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
     


In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
     

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:

model
     

In [14]:
model = model.to('cuda')

In [38]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)
     

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

# Data Prepration for Model Processing

In [12]:
X = list(data["text"])
y = list(data["toxic"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
     

In [ ]:
X

In [13]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:

print(X_train_tokenized['attention_mask'][0])

In [14]:
len(X_train),len(X_val)

(800, 200)

In [15]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])
     

In [16]:

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)
     

In [ ]:
train_dataset[5]


In [17]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
     

# Defining The Trainer

In [18]:

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=32

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
     

In [19]:

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=250, training_loss=0.12512020874023438, metrics={'train_runtime': 473.0909, 'train_samples_per_second': 16.91, 'train_steps_per_second': 0.528, 'total_flos': 1397777481600000.0, 'train_loss': 0.12512020874023438, 'epoch': 10.0})

In [20]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.7907278537750244,
 'eval_accuracy': 0.88,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 7.0023,
 'eval_samples_per_second': 28.562,
 'eval_steps_per_second': 3.57,
 'epoch': 10.0}

In [22]:
np.set_printoptions(suppress=True)

In [26]:
text = "That was good point"
#text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.2240, -2.7925]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.9976, 0.0024]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.9975677 , 0.00243228]], dtype=float32)

In [28]:
trainer.save_model('/content/gdrive/MyDrive/Short Text Claasification/Toxic_detect_Model')
     

In [33]:
model_2 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Short Text Claasification/Toxic_detect_Model")
#model_2.to('cuda')


In [47]:
#text = "That was good point"
text = "The non white homelands have so much trash and crime in them that non whites mainly choose to stay away ."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predicted_label_index = np.argmax(predictions)

if predicted_label_index == 0:
  print("Non - Toxic")
else:
  print("Toxic")

Toxic
